# DIGIT Local Setup - PGR Configuration

This notebook helps you configure PGR (Public Grievance Redressal) services on your local DIGIT setup.

## Prerequisites
- All DIGIT services running via `docker compose up` or `tilt up`
- Services healthy (check Tilt UI at http://localhost:10350)

## What this notebook does:
1. Connects to your local DIGIT services
2. Loads tenant and branding configuration
3. Sets up boundaries (administrative divisions)
4. Configures complaint types (ServiceDefs)
5. Creates test employees

---

## Step 1: Configuration

Update these values for your local setup:

In [ ]:
# =============================================================================
# CONFIGURATION - Update these for your environment
# =============================================================================

# Local DIGIT Gateway URL (Kong proxy)
# If running in Docker: use host.docker.internal or the Kong container name
# If running outside Docker: use localhost with Kong port
DIGIT_URL = "http://host.docker.internal:18000"  # Kong proxy port

# Alternative URLs to try if the above doesn't work:
# DIGIT_URL = "http://kong:8000"  # Inside Docker network
# DIGIT_URL = "http://localhost:18000"  # From host machine

# Admin credentials (default for local setup)
USERNAME = "ADMIN"
PASSWORD = "eGov@123"

# Target tenant
TENANT_ID = "pg"

# Template files (in ./templates/ directory)
TENANT_FILE = "templates/Tenant And Branding Master.xlsx"
BOUNDARY_FILE = "templates/Boundary_Master.xlsx"
COMMON_FILE = "templates/Common and Complaint Master.xlsx"

print(f"Configuration:")
print(f"  DIGIT URL: {DIGIT_URL}")
print(f"  Tenant: {TENANT_ID}")
print(f"  Username: {USERNAME}")

## Step 2: Test Connection

Let's verify we can connect to the DIGIT services:

In [ ]:
import requests
import json

def test_service(name, url, expected_in_response=None):
    """Test if a service is accessible"""
    try:
        resp = requests.get(url, timeout=5)
        if resp.status_code == 200:
            if expected_in_response:
                if expected_in_response in resp.text:
                    print(f"✅ {name}: OK")
                    return True
                else:
                    print(f"⚠️  {name}: Response unexpected")
                    return False
            print(f"✅ {name}: OK")
            return True
        else:
            print(f"❌ {name}: HTTP {resp.status_code}")
            return False
    except Exception as e:
        print(f"❌ {name}: {e}")
        return False

print("Testing DIGIT Services...\n")

# Test via Kong gateway
services_ok = True
services_ok &= test_service("Kong Gateway", f"{DIGIT_URL}/")

# If Kong doesn't work, try direct service URLs
if not services_ok:
    print("\nTrying direct service URLs...")
    DIRECT_URL = "http://host.docker.internal"
    test_service("MDMS (direct)", f"{DIRECT_URL}:18094/mdms-v2/health")
    test_service("User (direct)", f"{DIRECT_URL}:18107/user/health")
    test_service("PGR (direct)", f"{DIRECT_URL}:18083/pgr-services/health")
    print("\n⚠️  Kong not accessible. You may need to use direct URLs.")
else:
    print("\n✅ Kong gateway accessible - all services should work!")

## Step 3: Initialize CRS Loader

Import and initialize the data loader:

In [ ]:
from crs_loader import CRSLoader

# Initialize loader
loader = CRSLoader(DIGIT_URL)

# Login
print(f"Logging in as {USERNAME}...")
success = loader.login(username=USERNAME, password=PASSWORD, tenant_id=TENANT_ID)

if success:
    print(f"✅ Logged in successfully!")
    print(f"   User: {loader.uploader.user_info.get('userName')}")
    print(f"   Tenant: {loader.tenant_id}")
else:
    print("❌ Login failed. Check credentials and DIGIT_URL.")

## Step 4: Load Tenant & Branding (Phase 1)

This sets up the tenant configuration and UI branding:

In [ ]:
import os

if os.path.exists(TENANT_FILE):
    print(f"Loading tenant config from: {TENANT_FILE}")
    result = loader.load_tenant(TENANT_FILE, target_tenant=TENANT_ID)
    print("\n📋 Results:")
    for key, value in result.items():
        if isinstance(value, dict):
            created = value.get('created', 0)
            exists = value.get('exists', 0)
            print(f"   {key}: created={created}, exists={exists}")
else:
    print(f"⚠️  Template not found: {TENANT_FILE}")
    print("   Download templates from the CRS repository or create your own.")

## Step 5: Load Boundaries (Phase 2)

This creates the administrative boundary hierarchy (districts, wards, localities):

In [ ]:
if os.path.exists(BOUNDARY_FILE):
    print(f"Loading boundaries from: {BOUNDARY_FILE}")
    result = loader.load_boundaries(BOUNDARY_FILE, target_tenant=TENANT_ID, hierarchy_type="REVENUE")
    print("\n📋 Results:")
    print(f"   Status: {result.get('status')}")
    print(f"   Boundaries created: {result.get('boundaries_created', 0)}")
    print(f"   Relationships: {result.get('relationships_created', 0)}")
else:
    print(f"⚠️  Template not found: {BOUNDARY_FILE}")

## Step 6: Load Common Masters & Complaint Types (Phase 3)

This loads:
- Departments
- Designations  
- Complaint types (ServiceDefs for PGR)

In [ ]:
if os.path.exists(COMMON_FILE):
    print(f"Loading common masters from: {COMMON_FILE}")
    result = loader.load_common_masters(COMMON_FILE, target_tenant=TENANT_ID)
    print("\n📋 Results:")
    for key in ['departments', 'designations', 'complaint_types']:
        if result.get(key):
            created = result[key].get('created', 0)
            exists = result[key].get('exists', 0)
            print(f"   {key}: created={created}, exists={exists}")
else:
    print(f"⚠️  Template not found: {COMMON_FILE}")

## Step 7: Verify PGR Setup

Let's verify PGR is properly configured:

In [ ]:
print("Verifying PGR Configuration...\n")

# Check ServiceDefs
service_defs = loader.uploader.search_mdms_data(
    "RAINMAKER-PGR.ServiceDefs", 
    TENANT_ID, 
    limit=100
)

if service_defs:
    print(f"✅ Found {len(service_defs)} complaint types:")
    for svc in service_defs[:5]:  # Show first 5
        code = svc.get('serviceCode', 'N/A')
        name = svc.get('name', 'N/A')
        print(f"   - {code}: {name}")
    if len(service_defs) > 5:
        print(f"   ... and {len(service_defs) - 5} more")
else:
    print("❌ No complaint types found. Load Common Masters first.")

# Check boundaries
print("\nChecking boundaries...")
hierarchies = loader.uploader.search_boundary_hierarchies(TENANT_ID)
if hierarchies:
    print(f"✅ Found {len(hierarchies)} boundary hierarchies")
else:
    print("⚠️  No boundaries found. Load Boundaries first.")

print("\n" + "="*50)
print("PGR Setup Complete!")
print("="*50)
print(f"\nYou can now:")
print(f"  1. Access DIGIT UI: http://localhost:18080/digit-ui/")
print(f"  2. Test PGR API: POST http://localhost:18083/pgr-services/v2/request/_create")
print(f"  3. Use Kong gateway: http://localhost:18000/pgr-services/v2/request/_create")

## Optional: Rollback Data

If you need to reset and try again:

In [ ]:
# Uncomment to rollback:

# print("Rolling back common masters...")
# loader.rollback_common_masters(TENANT_ID)

# print("Deleting boundaries...")
# loader.delete_boundaries(TENANT_ID)

# print("Rolling back tenant...")
# loader.rollback_tenant(TENANT_ID)

print("Rollback commands are commented out. Uncomment to run.")

---

## Next Steps

1. **Create a test complaint** via the DIGIT UI or API
2. **Load employees** using the Employee template (optional)
3. **Customize complaint types** by editing the Excel template

For more details, see the [CRS DataLoader documentation](https://gist.github.com/ChakshuGautam/df762657f098acd4a3f46338bd2b32b4)